# __NB02 - Data Processing__

**Objective:** This notebook processes and cleans the data collected from notebook 1 and filters the necessary data from the large V-Dem dataset. This data processing produce the following dataframes:

- `vdem_filtered_df`
- `worldbank_df`
- `gdelt_df`

These dataframes will then each be exported as tables into the `processed_data` database with SQLite3.

**Imports:**

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
import os
import json
import functions

**Create Engine for SQL Database:**

In [3]:
engine = create_engine('sqlite:///../data/processed/processed_database.db', echo=False, isolation_level="AUTOCOMMIT")
with engine.connect() as conn:
    pass

# 1. Extract the data from the V-Dem dataset 

1.1 Read the V-Dem dataset as a `pandas` dataframe

In [4]:
vdem_df = pd.read_csv("../data/processed/V-Dem-CY-Full+Others-v14.csv") # avoid printing the df as it is an extremely large dataset

/var/folders/w0/cyh7_1d94mx_ft1mfdkmbwrw0000gn/T/ipykernel_4371/2454491280.py:1: DtypeWarning: Columns (364,365,366,399,415,804,836,837,924,1240,1257,1486,3094,3168,3169,3341,3342,3344,3345,3347,3350,3352) have mixed types. Specify dtype option on import or set low_memory=False.
  vdem_df = pd.read_csv("../data/processed/V-Dem-CY-Full+Others-v14.csv") # avoid printing the df as it is an extremely large dataset


1.2 Filter out the dataset to only contain data for the US, Brazil, India, Iran, France, and South Africa from 2014-2023 and for the following indicators:
- `v2x_polyarchy` - electoral democracy index
- `v2peapsecon` - access to public services distributed by socio-economic position
- `v2pepwrses` - power distributed by socioeconomic position
- `v2mecrit` - print/broadcast media critical
- `v2cacamps` - political polairsation

In [5]:
interesting_columns = ['country_name', 'country_text_id', 'year', 'v2x_polyarchy', 'v2peapsecon', 'v2pepwrses', 'v2mecrit', 'v2cacamps']

required_countries = ['United States of America', 'Brazil', 'India', 'Iran', 'France', 'South Africa']

vdem_filtered_df = vdem_df[
    (vdem_df['year'] >= 2014) & 
    (vdem_df['year'] <= 2023) &
    (vdem_df['country_name'].isin(required_countries))
    ][interesting_columns]

1.3 Change the `country_text_id` from ALPHA-3 to ALPHA-2

In [6]:
vdem_filtered_df['country_text_id'] = vdem_filtered_df['country_text_id'].str[:2]

# 2. Add the V-Dem dataset to a database

2.1 creating a table in the `processed_data.db` using SQLite for the vdem_filtered_df named `vdem_data`

primary keys:
 - `country_id`
 - `year`

In [ ]:
create_statement_vdem = """
    CREATE TABLE vdem_data (
        country_name CHAR(24),
        country_text_id CHAR(2),
        year INTEGER,
        v2x_polyarchy REAL,
        v2peapsecon REAL,
        v2pepwrses REAL,
        v2mecrit REAL,
        v2cacamps REAL,
        PRIMARY KEY (country_text_id, year)
        );
    """

with engine.connect() as conn:
    conn.execute(text(create_statement_vdem))
    vdem_filtered_df.to_sql("vdem_data", conn, if_exists="append", index=False)

# 3. Clean the World Bank dataset 

3.1 load all raw files and use pandas to concatinate them into one pandas dataframe which filters for the following columns: 

- `country_id` - country ID
- `country_name` - the name of the country (US, Brazil, India, Iran, France, and South Africa)
- `countryiso3code` - country code 
- `indicator.id` - indicator ID
- `indicator.value` - name of indicator 
- `date` - in years (2014-2023)
- `value` - numeric value of indictor 
- `unit` - unit of value 


In [11]:
data_directory = "../data/raw/world_bank"
data_files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.endswith(".json")]
worldbank_df = pd.concat((functions.process_file(file) for file in data_files), ignore_index=True)
worldbank_df = worldbank_df.rename(columns={
    'country.id': 'country_id',
    'country.value': 'country_name',
    'countryiso3code': 'iso3code',
    'indicator.id': 'indicator_id',
    'indicator.value': 'indicator_name',
    'date': 'year',
    'value': 'value',
    'unit': 'unit'
})[["country_id", "country_name", "iso3code", "indicator_id", "indicator_name", "year", "value", "unit"]]

3.2 remove empty columns and ensure data types are appropriate

In [12]:
worldbank_df['year'] = pd.to_numeric(worldbank_df['year'], errors='coerce').astype('Int64')
worldbank_df['value'] = pd.to_numeric(worldbank_df['value'], errors='coerce').astype(float) # Ensure 'value' is a float
worldbank_df = worldbank_df.where(pd.notnull(worldbank_df), None)
worldbank_df.drop_duplicates(inplace=True)
worldbank_df.drop(columns=['unit'], inplace=True)

# 4. Add the World Bank data to the database 

4.1 creating a table in the `processed_data.db` using SQLite for the `worldbank_df` named `world_bank`

primary keys:
 - `country_id`
 - `year`
 - `indicator_id`

In [13]:
create_statement_world_bank = """
    CREATE TABLE world_bank (
        country_id CHAR(2),
        country_name CHAR(24),
        iso3code CHAR(3),
        indicator_id CHAR(50),
        indicator_name CHAR(50),
        year INTEGER,
        value REAL,
        PRIMARY KEY (country_id, year, indicator_id)
        );
    """

with engine.connect() as conn:
    conn.execute(text(create_statement_world_bank))
    worldbank_df.to_sql("world_bank", conn, if_exists="append", index=False)

# 5. Create a Smaller GDELT dataset 

As the full GDELT data set is too large to load to a SQLite database or efficiently explore, we aggregated the data to improve its usability. 
instead of a column for each event, the final `gdelt_df` contains the following:
- `country_code` and `year`: from the original data set
- `event_code`: code categorizing a specific event-- see Notebook 1 for information on the codes' meanings  
- `num_occurences`: number of corrences of each event code (per country, per year)
- `num_mentions`: number of mentions in the press for each event code (per country, per year)
- `avg_goldstein`, `min_goldstein`, `max_goldstein`: summary statistics for the golstein score (per event_code, per country, per year)
- `avg_tone`, `min_tone`, `max_tone`: summary statistics for the tone of each event (per event_code, per country, per year)

In [14]:
data_directory = "../data/raw/gdelt"
gdelt_df = functions.process_gdelt_json(data_directory)

# 6. Add Cleaned GDELT to the Data Base

In [15]:
create_statement_gdelt = """
    CREATE TABLE gdelt_events (
        country_code CHAR(3),
        year INTEGER,
        EventCode INTEGER,
        num_occurrences INTEGER,
        num_mentions INTEGER,
        avg_goldstein REAL,
        min_goldstein REAL,
        max_goldstein REAL,
        avg_tone REAL,
        min_tone REAL,
        max_tone REAL,
        PRIMARY KEY (country_code, year, EventCode)
        );
    """

with engine.connect() as conn:
    conn.execute(text(create_statement_gdelt))
    gdelt_df.to_sql("gdelt_events", conn, if_exists="append", index=False)